OCR WITH KERAS

In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imutils import build_montages
from PIL import Image

from models import ResNet
import matplotlib
matplotlib.use("Agg")

EPOCHS = 50
INIT_LR = 1e-1
BS = 128

Datasets:

A-Z
http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

Load A-Z dataset:

In [2]:
def load_a_z_font_dataset(rootdir):
    data = []
    labels = []
    
    for (dirpath, dirnames, filenames) in os.walk(rootdir):
        for filename in filenames:
            labels.append(int(filename[3:6])-1)
            
            imageFile = Image.open(os.path.join(dirpath, filename))
            image = np.array(imageFile.getdata(), dtype='uint8')
            image = image.reshape((128, 128))
            data.append(image)                    

    return (data, labels)

Pre-processing:

In [3]:
(data, labels) = load_a_z_font_dataset(r'C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\datasets\a_z_font_maiusculas')

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

data = np.expand_dims(data, axis=-1)
data /= 255.0

le = LabelBinarizer()

labels = le.fit_transform(labels)
ounts = labels.sum(axis=0)

Training assesment balance:

In [4]:
classTotals = labels.sum(axis=0)
classWeight = {}

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

(trainX, testX, trainY, testY) = train_test_split(data,
                                                  labels, test_size=0.20, stratify=None, random_state=42) #VER COMO USAR O STRATIFY

Preparing to train:

In [5]:
aug = ImageDataGenerator(rotation_range=10, zoom_range=0.05, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15, horizontal_flip=False, fill_mode="nearest")

print("[INFO] compiling model...")

opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
                     (64, 64, 128, 256), reg=0.0005)
                     
model.compile(loss="categorical_crossentropy",
              optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


C:\Users\alanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Training:

In [8]:
print("[INFO] training network...")

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS), 
            validation_data=(testX, testY), 
            #steps_per_epoch=len(trainX),
            epochs=EPOCHS,
            class_weight=classWeight,
            verbose=1)

[INFO] training network...
Epoch 1/50
166/166 [==============================] - 235s 1s/step - loss: 2.7988 - accuracy: 0.3278 - val_loss: 1.9773 - val_accuracy: 0.6734
Epoch 2/50
166/166 [==============================] - 233s 1s/step - loss: 1.5497 - accuracy: 0.7266 - val_loss: 1.0967 - val_accuracy: 0.8090
Epoch 3/50
166/166 [==============================] - 236s 1s/step - loss: 0.9752 - accuracy: 0.8486 - val_loss: 0.8216 - val_accuracy: 0.8550
Epoch 4/50
166/166 [==============================] - 238s 1s/step - loss: 0.7328 - accuracy: 0.8981 - val_loss: 0.6619 - val_accuracy: 0.9129
Epoch 5/50
166/166 [==============================] - 237s 1s/step - loss: 0.6212 - accuracy: 0.9165 - val_loss: 0.5469 - val_accuracy: 0.9285
Epoch 6/50
166/166 [==============================] - 236s 1s/step - loss: 0.5617 - accuracy: 0.9274 - val_loss: 0.5213 - val_accuracy: 0.9317
Epoch 7/50
166/166 [==============================] - 236s 1s/step - loss: 0.5223 - accuracy: 0.9338 - val_loss: 0.

Network evaluation:

In [9]:
labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=labelNames))


[INFO] evaluating network...
              precision    recall  f1-score   support

           A       0.98      0.99      0.99       191
           B       0.94      0.97      0.96       188
           C       0.99      0.92      0.96       226
           D       0.99      0.97      0.98       225
           E       0.94      0.97      0.96       194
           F       0.95      0.97      0.96       203
           G       0.95      0.95      0.95       216
           H       0.98      0.98      0.98       193
           I       0.95      0.94      0.94       186
           J       0.96      0.96      0.96       219
           K       0.98      0.98      0.98       212
           L       0.99      0.97      0.98       194
           M       0.97      0.99      0.98       210
           N       0.99      0.95      0.97       188
           O       0.99      0.99      0.99       213
           P       0.96      0.97      0.97       210
           Q       0.96      0.97      0.97       19

Saving the model and training history:

In [10]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_only_capital.model"
model.save(model_path, save_format="h5")

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Trainning Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


Testing:

In [13]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_only_capital.model"
model = load_model(model_path)

images = [] 
for i in np.random.choice(np.arange(0, len(testY)), size=(49,)):
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]

    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)

    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)

    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

    images.append(image)

montage = build_montages(images, (96, 96), (7, 7))[0]

cv2.imshow("OCR Results", montage)
cv2.waitKey(0)

-1

Predict:

In [59]:
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import numpy as np
import imutils
import cv2

model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_only_capital.model"

print("[INFO] loading OCR model...")
model = load_model(model_path)
print(model_path)

image_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\images\placa-perigo.png"

image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

edged = cv2.Canny(blurred, 30, 150)

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]

chars = []

for c in cnts:
	(x, y, w, h) = cv2.boundingRect(c)

	if (w >= 20 and w <= 150) and (h >= 15 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		thresh = cv2.threshold(roi, 0, 255,
			cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
		(tH, tW) = thresh.shape

		if tW > tH:
			thresh = imutils.resize(thresh, width=32)
		else:
			thresh = imutils.resize(thresh, height=32)

		(tH, tW) = thresh.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)

		padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(255, 255, 255))
		
		padded = cv2.resize(padded, (32, 32))
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)

		chars.append((padded, (x, y, w, h)))

boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")

preds = model.predict(chars)

labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

for (pred, (x, y, w, h)) in zip(preds, boxes):
	i = np.argmax(pred)
	prob = pred[i]
	if prob > 0.8:
		label = labelNames[i]

		print("[INFO] {} - {:.2f}%".format(label, prob * 100))
		cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
		cv2.putText(image, label, (x - 10, y - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

cv2.imshow("Image", image)
cv2.waitKey(0)

[INFO] loading OCR model...
C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_only_capital.model
[INFO] A - 99.57%
[INFO] F - 99.22%
[INFO] A - 99.71%
[INFO] S - 99.13%
[INFO] T - 91.55%
[INFO] E - 99.81%
[INFO] S - 98.98%
[INFO] E - 99.70%


-1